In [2]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.debugger import DbgMode
from debugger.bench_viewer import BenchViewer
from debugger.debugger_options import DebugOptions
from debugger.strainer import StrainerStatus
from benchmark_consts import *

In [3]:
qs = UNSTABLE_MARIPOSA
options = DebugOptions()

mv = BenchViewer(qs, options)

[WARN] [init] 5b7d18ab3e trace did not fail, fallback to fast_fail [WARN] [init] 2a3b1202bd trace slow unknown, fallback to timeout 

[WARN] [init] e998d08b31 trace slow unknown, fallback to timeout 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proofs available 
[WARN] [init] 0b497bc090 trace did not fail, fallback to fast_fail 
[WARN] [init] d4e5e77cbc trace slow unknown, fallback to timeout 
[WARN] [init] 3cead21983 trace slow unknown, fallback to timeout 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proofs available 
[WARN] [init] 3ce5c5e4dc trace slow unknown, fallback to timeout 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proofs available 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proofs available 


In [4]:
mv.status.print_status()

| category                   |   count | percentage   |
|----------------------------|---------|--------------|
| StrainerStatus.FINISHED    |     535 | 98.17 %      |
| StrainerStatus.NOT_CREATED |       6 | 1.1 %        |
| StrainerStatus.NO_PROOF    |       4 | 0.73 %       |
| total                      |     545 | 100.00 %     |


In [6]:
from utils.analysis_utils import fmt_percent

bench_total = len(mv.status.tally)
bench_finished = len(mv.status[StrainerStatus.FINISHED])
# to_fixable = 0
to_cases = dict()
ff_cases = dict()

def print_break_down(p, q):
    return f"{p}/{q} ({fmt_percent(p, q, 1)})"

for q in mv.status[StrainerStatus.FINISHED]:
    dbg = mv[q]
    if dbg.mode == DbgMode.TIMEOUT:
        to_cases[q] = q in mv.fixable
    elif dbg.mode == DbgMode.FAST_FAIL:
        ff_cases[q] = q in mv.fixable
    else:
        print("Unexpected mode", dbg.mode)
        assert False

to_fixable = sum(to_cases.values())
ff_fixable = sum(ff_cases.values())

print("Test Finished:", print_break_down(bench_finished, bench_total))
print("\tTOs:", print_break_down(len(to_cases), bench_finished))
print("\t\tTO fixable:", print_break_down(to_fixable, len(to_cases)))
print("\tFFs:", print_break_down(len(ff_cases), bench_finished))
print("\t\tFF fixable:", print_break_down(ff_fixable, len(ff_cases)))

print("Test Not Finished:", print_break_down(len(mv.status[StrainerStatus.NOT_CREATED]), bench_total))
print("No Proofs:", fmt_percent(len(mv.status[StrainerStatus.NO_PROOF]), bench_total))
print("* fixable:", print_break_down(to_fixable + ff_fixable, bench_total))

Test Finished: 535/545 (98.2%)
	TOs: 444/535 (83.0%)
		TO fixable: 276/444 (62.2%)
	FFs: 91/535 (17.0%)
		FF fixable: 51/91 (56.0%)
Test Not Finished: 6/545 (1.1%)
No Proofs: 0.73%
* fixable: 327/545 (60.0%)


In [6]:
for q in mv.status[StrainerStatus.NOT_CREATED]:
    print("./src/debugger3.py -m auto -i", mv[q].name_hash, "--create-project")

./src/debugger3.py -m auto -i ab307c9008 --create-project
./src/debugger3.py -m auto -i d4e5e77cbc --create-project
./src/debugger3.py -m auto -i 2a3b1202bd --create-project
./src/debugger3.py -m auto -i 3ce5c5e4dc --create-project
./src/debugger3.py -m auto -i 3cead21983 --create-project
./src/debugger3.py -m auto -i e998d08b31 --create-project


In [7]:
# for q in mv.unfixable:
#     print(q)
print("Unfixable:", len(mv.unfixable))
unfixable = set([mv[q].name_hash for q in mv.unfixable])
print(len(unfixable & set(DOUBLETON_TARGETS)))

Unfixable: 208
114


In [27]:
for q in mv.status[StrainerStatus.NOT_TESTED]:
    print("./src/carve_and_rerun.py", mv[q].strainer.test_dir)

In [7]:
from utils.cluster_utils import get_sync_commands

for q in mv.fixable:
    dbg = mv[q]
    dbg.collect_garbage()

with open("miscs/send.sh", "w") as f:
    f.write("#!/bin/bash\n")
    cmds = get_sync_commands("g2001", mv.get_sync_dirs())
    for cmd in cmds:
        f.write(cmd + "\n")
    f.write("echo 'done'\n")